In [1]:
import json
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import regex as re
from nltk.corpus import stopwords

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import datetime

import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

import random
import os

from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from keras import optimizers
from keras import backend as K

import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("__file__"))))
import train_test as tt

2023-04-24 15:00:03.621085: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 15:00:03.721837: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-24 15:00:03.724824: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-24 15:00:03.724838: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
    
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [3]:
################################### 논문 재현(변수 다르고, Amazon 사용) #######################

In [4]:
# 데이터 생성, 처리
def time_conversion(time):
    time = pd.Timestamp(time)
    return time

def date_cut_day(dataframe_date_timestamp):
    dataframe_date_timestamp = datetime.datetime.strftime(dataframe_date_timestamp, "%Y-%m-%d")
    return dataframe_date_timestamp

def train_test_result(dir, stock, variable, window_size, start_date, end_date):
    #score data 뽑기
    score = pd.read_csv(dir)
    score.columns = ["Date", "N_Score", "B_Score"]
    
    end_1_next_date = pd.to_datetime(end_date, errors="ignore") + datetime.timedelta(days=1)
    end_1_next_date = end_1_next_date.strftime('%Y-%m-%d')
    
    #price data 뽑기
    price_data = yf.download([stock],start = start_date, end = end_1_next_date)
    
    opening = price_data.index.copy()
    opening = pd.DataFrame(opening)
    opening["opening_date"] = 1
    opening.Date = opening.Date.apply(lambda x: date_cut_day(x))
    
    start_2_previous_date = pd.to_datetime(start_date, errors="ignore") - datetime.timedelta(days=2)
    start_2_previous_date = start_2_previous_date.strftime('%Y%m%d')
    
    Date = pd.date_range(start=start_2_previous_date, end=pd.to_datetime(end_date).strftime('%Y%m%d'))
    Date = pd.DataFrame({"Date" : Date.values})
    Date.Date = Date.Date.apply(lambda x: date_cut_day(x))

    
    set_news_data_date = pd.merge(Date, opening, how ="left",left_on='Date', right_on = "Date") 
    set_news_data_date = set_news_data_date.where(pd.notnull(set_news_data_date), 0) 
    set_news_data_date.Date = set_news_data_date.Date.apply(lambda x: pd.to_datetime(x, errors="ignore"))

    standard=set_news_data_date.Date.iloc[len(set_news_data_date)-1] + datetime.timedelta(days=1)
    set_news_data_date["price_date"]=0
    for i in range(len(set_news_data_date)-1,-1,-1):
        if i==(len(set_news_data_date)-1):
            standard = set_news_data_date.Date[i]
            set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)
        elif (set_news_data_date.opening_date[i]==1)&(set_news_data_date.opening_date[i+1]==1):
            standard = set_news_data_date.Date[i]
            set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)
        elif (i!=0):
            if((set_news_data_date.opening_date[i]==1)&(set_news_data_date.opening_date[i+1]==0)&(set_news_data_date.opening_date[i-1]==0)):
                set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)
                standard = set_news_data_date.Date[i]
            else:
                set_news_data_date.price_date[i]=standard
        else:
            set_news_data_date.price_date[i]=standard
    
    set_news_data_date.Date = set_news_data_date.Date.apply(lambda x: date_cut_day(x))
    
    score = pd.merge(set_news_data_date,score, how ="left",on = "Date")
    
    score = score[['Date','N_Score', "B_Score", "price_date"]]
    score = score.groupby('price_date').mean({"N_Score", "B_Score"})
    end_1_previous_date = pd.to_datetime(end_date, errors="ignore") - datetime.timedelta(days=1)
    end_1_previous_date = end_1_previous_date.strftime('%Y-%m-%d')
    score = score[score.index<=end_1_previous_date]

    start_2_previous_date = pd.to_datetime(start_date, errors="ignore") - datetime.timedelta(days=2)
    start_2_previous_date = start_2_previous_date.strftime('%Y-%m-%d')
    first_data = yf.download([stock],start = start_2_previous_date, end = start_date)
    
    price_data = price_data.reset_index()
    score = score.reset_index()
    score.columns = ["Date", "N_Score", "B_Score"]
    
    #price data와 score data 결합
    price_data = pd.merge(price_data, score, how ="left",left_on='Date', right_on = "Date")[["Close","Open", "High" , "Low", "Volume" ,"N_Score", "B_Score"]]

    #뉴스가 없는 날은 즉, score가 없는 날은 중립의 의미로 0으로 처리
    price_data.N_Score[price_data.N_Score.isnull()]=0
    price_data.B_Score[price_data.B_Score.isnull()]=0

    #전날 종가의 영향을 받으므로 전날 종가 변수를 생성
    price_data["before_close"] = 0
    price_data = price_data.reset_index(drop=True)
    for i in range(len(price_data)-1):
        price_data.before_close[i+1] = price_data.Close[i]
    
    price_data.before_close[0] = first_data["Close"][0]

    # minmaxscaler 사용
    price_data.columns = ["Close","Open", "High" , "Low", "Volume" ,"N_Score", "B_Score","before_close"]
    
    x = price_data[variable]
    y = price_data[["Close"]]
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    x = pd.DataFrame(scaler_x.fit_transform(x))
    y = pd.DataFrame(scaler_y.fit_transform(y))
    
    x.columns = variable
    y.columns = ["Close"]

    train_index = int(len(x)*0.8)

    train_x = x.iloc[0:train_index]
    test_x = x.iloc[train_index:len(x)]

    train_y = y.iloc[0:train_index]
    test_y = y.iloc[train_index:len(y)]

    train_x = train_x.to_numpy().reshape(train_x.shape[0],1,train_x.shape[1])
    train_y = train_y.to_numpy().reshape(train_y.shape[0],train_y.shape[1])
    test_x = test_x.to_numpy().reshape(test_x.shape[0],1,test_x.shape[1])
    test_y = test_y.to_numpy().reshape(test_y.shape[0],test_y.shape[1])
    
    #window size에 맞게 데이터 설정
    x = np.zeros(shape=(train_x.shape[0]-window_size+1,window_size,x.shape[1]))
    for i in range(train_x.shape[0]-window_size+1):
        x[i]=np.vstack((train_x[i:i+window_size]))

    y = train_y[window_size-1:train_x.shape[0]]

    x_t = np.zeros(shape=(test_x.shape[0]-window_size+1,window_size,x.shape[2]))
    for i in range(test_x.shape[0]-window_size+1):
        x_t[i]=np.vstack((test_x[i:i+window_size]))
    
    y_t = test_y[window_size-1:test_x.shape[0]]
    
    return scaler_x, scaler_y, x, y, x_t, y_t

In [5]:
dir = "nyt_score.csv"
stock = "AMZN"
variable = ["Open","High","Low","Volume","N_Score", "B_Score"]
window_size = 1
start_date = "2015-01-01"
end_date = "2020-08-13"

scaler_x, scaler_y, x, y, x_t, y_t = train_test_result(dir, stock, variable, window_size, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_2758964/4182547383.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)


[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_2758964/4182547383.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.before_close[i+1] = price_data.Close[i]


In [6]:
lstm_1_model = Sequential()
lstm_1_model.add(LSTM(50, input_shape = (x.shape[1],x.shape[2]),return_sequences=True))
lstm_1_model.add((LSTM(50)))
lstm_1_model.add(Dense(1,activation="tanh"))
lstm_1_model.compile(optimizer="adam", loss = 'mse' , metrics=["mae"])
history = lstm_1_model.fit(x, y, epochs=40, batch_size=128, validation_split=0.2)
    
lstm_1_model.evaluate(x_t, y_t)

2023-04-24 15:00:11.660138: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-24 15:00:11.660266: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-24 15:00:11.660348: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-24 15:00:11.660427: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-04-24 15:00:11.693961: W tensorfl

Epoch 1/40
8/8 [==============================] - 3s 97ms/step - loss: 0.0466 - mae: 0.1743 - val_loss: 0.2056 - val_mae: 0.4508
Epoch 2/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0319 - mae: 0.1345 - val_loss: 0.1567 - val_mae: 0.3928
Epoch 3/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0207 - mae: 0.1049 - val_loss: 0.1089 - val_mae: 0.3263
Epoch 4/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0143 - mae: 0.0914 - val_loss: 0.0692 - val_mae: 0.2584
Epoch 5/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0130 - mae: 0.0940 - val_loss: 0.0526 - val_mae: 0.2244
Epoch 6/40
8/8 [==============================] - 0s 10ms/step - loss: 0.0120 - mae: 0.0911 - val_loss: 0.0526 - val_mae: 0.2250
Epoch 7/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0103 - mae: 0.0826 - val_loss: 0.0523 - val_mae: 0.2250
Epoch 8/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0088 - mae: 0.0753 - val

[0.001842974335886538, 0.02323918230831623]

In [7]:
lstm_1_model.evaluate(x_t, y_t)

9/9 [==============================] - 0s 3ms/step - loss: 0.0018 - mae: 0.0232


[0.001842974335886538, 0.02323918230831623]

In [8]:
lstm_1_pred = lstm_1_model.predict(x_t)
lstm_1_pred = scaler_y.inverse_transform(lstm_1_pred)
y_t = scaler_y.inverse_transform(y_t)
root_mean_squared_error(y_t, lstm_1_pred), mean_squared_error(y_t, lstm_1_pred), mean_absolute_error(y_t, lstm_1_pred), MAPE(y_t, lstm_1_pred)

9/9 [==============================] - 1s 2ms/step


(<tf.Tensor: shape=(), dtype=float64, numpy=6.306505209145918>,
 39.772007952984595,
 3.4138929549038615,
 2.594839723514224)

In [ ]:
##################### 논문 재현 DowJ ##########################

In [9]:
dir = "nyt_score.csv"
stock = "^DJI"
variable = ["Open","High","Low","Volume","N_Score", "B_Score"]
window_size = 1
start_date = "2015-01-01"
end_date = "2020-08-13"

scaler_x, scaler_y, x, y, x_t, y_t = train_test_result(dir, stock, variable, window_size, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_2758964/4182547383.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)


[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_2758964/4182547383.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.before_close[i+1] = price_data.Close[i]


In [10]:
lstm_1_model = Sequential()
lstm_1_model.add(LSTM(50, input_shape = (x.shape[1],x.shape[2]),return_sequences=True))
lstm_1_model.add((LSTM(50)))
lstm_1_model.add(Dense(1,activation="tanh"))
lstm_1_model.compile(optimizer="adam", loss = 'mse' , metrics=["mae"])
history = lstm_1_model.fit(x, y, epochs=40, batch_size=128, validation_split=0.2)
    
lstm_1_model.evaluate(x_t, y_t)

Epoch 1/40
8/8 [==============================] - 3s 82ms/step - loss: 0.1285 - mae: 0.2944 - val_loss: 0.4210 - val_mae: 0.6456
Epoch 2/40
8/8 [==============================] - 0s 11ms/step - loss: 0.1010 - mae: 0.2506 - val_loss: 0.3415 - val_mae: 0.5809
Epoch 3/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0733 - mae: 0.2020 - val_loss: 0.2518 - val_mae: 0.4980
Epoch 4/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0462 - mae: 0.1525 - val_loss: 0.1546 - val_mae: 0.3887
Epoch 5/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0261 - mae: 0.1299 - val_loss: 0.0761 - val_mae: 0.2699
Epoch 6/40
8/8 [==============================] - 0s 10ms/step - loss: 0.0196 - mae: 0.1250 - val_loss: 0.0396 - val_mae: 0.1918
Epoch 7/40
8/8 [==============================] - 0s 11ms/step - loss: 0.0185 - mae: 0.1229 - val_loss: 0.0339 - val_mae: 0.1773
Epoch 8/40
8/8 [==============================] - 0s 10ms/step - loss: 0.0150 - mae: 0.1103 - val

[0.003818240249529481, 0.05162724480032921]

In [11]:
lstm_1_model.evaluate(x_t, y_t)

9/9 [==============================] - 0s 2ms/step - loss: 0.0038 - mae: 0.0516


[0.003818240249529481, 0.05162724480032921]

In [12]:
lstm_1_pred = lstm_1_model.predict(x_t)
lstm_1_pred = scaler_y.inverse_transform(lstm_1_pred)
y_t = scaler_y.inverse_transform(y_t)
root_mean_squared_error(y_t, lstm_1_pred), mean_squared_error(y_t, lstm_1_pred), mean_absolute_error(y_t, lstm_1_pred), MAPE(y_t, lstm_1_pred)

9/9 [==============================] - 1s 3ms/step


(<tf.Tensor: shape=(), dtype=float64, numpy=858.366089725644>,
 736792.3439908923,
 717.166195064046,
 2.6650830226510496)

In [13]:
#################################### 2 variable(news, close 사용) #####################################

In [19]:
dir = "/home/whfhrs3260/csv_data/price_data_score_10years.csv"
stock = "^DJI"
variable = ["before_close","Score"]
window_size = 1
start_date = "2015-01-01"
end_date = "2020-08-13"

scaler_x, scaler_y, x, y, x_t, y_t = tt.train_test_result(dir, stock, variable, window_size, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


/home/whfhrs3260/Python/10year_data/review/train_test.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)


[*********************100%***********************]  1 of 1 completed


/home/whfhrs3260/Python/10year_data/review/train_test.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.before_close[i+1] = price_data.Close[i]


In [20]:
lstm_1_model = Sequential()
lstm_1_model.add(LSTM(50, input_shape = (x.shape[1],x.shape[2]),return_sequences=True))
lstm_1_model.add((LSTM(50)))
lstm_1_model.add(Dense(1,activation="tanh"))
lstm_1_model.compile(optimizer="adam", loss = 'mse' , metrics=["mae"])
history = lstm_1_model.fit(x, y, epochs=40, batch_size=128, validation_split=0.2)
    
lstm_1_model.evaluate(x_t, y_t)

Epoch 1/40
7/7 [==============================] - 3s 99ms/step - loss: 0.0941 - mae: 0.2506 - val_loss: 0.4047 - val_mae: 0.6340
Epoch 2/40
7/7 [==============================] - 0s 12ms/step - loss: 0.0744 - mae: 0.2116 - val_loss: 0.3441 - val_mae: 0.5843
Epoch 3/40
7/7 [==============================] - 0s 12ms/step - loss: 0.0564 - mae: 0.1733 - val_loss: 0.2805 - val_mae: 0.5270
Epoch 4/40
7/7 [==============================] - 0s 13ms/step - loss: 0.0406 - mae: 0.1367 - val_loss: 0.2143 - val_mae: 0.4598
Epoch 5/40
7/7 [==============================] - 0s 13ms/step - loss: 0.0295 - mae: 0.1251 - val_loss: 0.1539 - val_mae: 0.3886
Epoch 6/40
7/7 [==============================] - 0s 13ms/step - loss: 0.0251 - mae: 0.1310 - val_loss: 0.1142 - val_mae: 0.3335
Epoch 7/40
7/7 [==============================] - 0s 13ms/step - loss: 0.0243 - mae: 0.1344 - val_loss: 0.1032 - val_mae: 0.3167
Epoch 8/40
7/7 [==============================] - 0s 14ms/step - loss: 0.0233 - mae: 0.1316 - val

[0.003308473387733102, 0.04681278020143509]

In [21]:
lstm_1_model.evaluate(x_t, y_t)

14/14 [==============================] - 0s 3ms/step - loss: 0.0033 - mae: 0.0468


[0.003308473387733102, 0.04681278020143509]

In [22]:
lstm_1_pred = lstm_1_model.predict(x_t)
lstm_1_pred = scaler_y.inverse_transform(lstm_1_pred)
y_t = scaler_y.inverse_transform(y_t)
root_mean_squared_error(y_t, lstm_1_pred), mean_squared_error(y_t, lstm_1_pred), mean_absolute_error(y_t, lstm_1_pred), MAPE(y_t, lstm_1_pred)

14/14 [==============================] - 1s 2ms/step


(<tf.Tensor: shape=(), dtype=float64, numpy=799.0145507663063>,
 638424.2523362823,
 650.2872426470589,
 2.465812723214691)